In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df_details = pd.read_csv('../data/desafio_details.csv')
df_priceav = pd.read_csv('../data/desafio_priceav.csv')

Calculando faturamento por anúncio

In [3]:
df_details['faturamento'] = 0
for i in range(len(df_details)):
    listing_id = df_details['airbnb_listing_id'].iloc[i]
    df_details['faturamento'].iloc[i] = df_priceav[df_priceav['airbnb_listing_id'] == listing_id]['price_string'].sum()

In [24]:
df_details

,Unnamed: 0,airbnb_listing_id,suburb,ad_name,number_of_bedrooms,number_of_bathrooms,star_rating,is_superhost,number_of_reviews,faturamento
0,0,31389869,Jurerê,Lindo Apartamento em Jurerê,2.0,2.0,5.0,False,15.0,13510.0
1,1,40010667,Canasvieiras,"Residencial Arruda, 1 quarto",1.0,1.0,NaN,False,0.0,35740.0
2,2,38905997,Ingleses,Apartamento NOVO Completo - Moderno e Sofisticado,1.0,1.0,4.5,True,13.0,16806.0
3,3,22343656,Ingleses,06- Apartamento 02 habitaciones,2.0,1.0,5.0,True,28.0,0.0
4,4,18328184,Canasvieiras,"Apto 2 quartos em Canasvieiras, Florianopolis!",2.0,1.0,5.0,True,35.0,26825.0
...,...,...,...,...,...,...,...,...,...,...
4686,4686,34392248,Ingleses,Lindo e acogedor apto en zona norte de la Isla,2.0,2.0,NaN,False,0.0,0.0
4687,4687,44538969,Canasvieiras,"Ótimo apto no centro de Canas, a 450m do mar",1.0,1.0,NaN,False,0.0,0.0
4688,4688,16609826,Jurerê,Agradável Apartamento em Jurerê Internacional,3.0,2.0,5.0,True,30.0,0.0
4689,4689,31239515,Canasvieiras,Cobertura no centro 3 dormitorios com AR.,3.0,2.0,NaN,True,2.0,0.0


analisando a correlação

In [4]:
df_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4691 entries, 0 to 4690
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           4691 non-null   int64  
 1   airbnb_listing_id    4691 non-null   int64  
 2   suburb               4691 non-null   object 
 3   ad_name              4691 non-null   object 
 4   number_of_bedrooms   4508 non-null   float64
 5   number_of_bathrooms  4690 non-null   float64
 6   star_rating          2570 non-null   float64
 7   is_superhost         4691 non-null   bool   
 8   number_of_reviews    4684 non-null   float64
 9   faturamento          4691 non-null   float64
dtypes: bool(1), float64(5), int64(2), object(2)
memory usage: 334.5+ KB


In [47]:
df_details.shape

(4691, 10)

In [46]:
df_details['faturamento'].corr(df_details['number_of_reviews'])

0.044415841443332466

In [7]:
df_details.dropna(inplace=True)
df_details.shape

(2450, 10)

In [8]:
df_details.corr()

,Unnamed: 0,airbnb_listing_id,number_of_bedrooms,number_of_bathrooms,star_rating,is_superhost,number_of_reviews,faturamento
Unnamed: 0,1.000000,0.007333,-0.012281,0.014472,0.010891,0.011288,-0.035585,-0.103082
airbnb_listing_id,0.007333,1.000000,-0.050457,-0.038661,-0.012835,0.012875,-0.270643,0.082133
number_of_bedrooms,-0.012281,-0.050457,1.000000,0.758107,0.005798,-0.108500,-0.125340,0.131234
number_of_bathrooms,0.014472,-0.038661,0.758107,1.000000,0.020738,-0.103450,-0.103849,0.155047
star_rating,0.010891,-0.012835,0.005798,0.020738,1.000000,0.196364,0.121407,0.053912
is_superhost,0.011288,0.012875,-0.108500,-0.103450,0.196364,1.000000,0.287413,0.062671
number_of_reviews,-0.035585,-0.270643,-0.125340,-0.103849,0.121407,0.287413,1.000000,0.039613
faturamento,-0.103082,0.082133,0.131234,0.155047,0.053912,0.062671,0.039613,1.000000


In [36]:
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = df_details.corr()
corr.style.background_gradient(cmap='coolwarm')

,Unnamed: 0,airbnb_listing_id,number_of_bedrooms,number_of_bathrooms,star_rating,is_superhost,number_of_reviews,faturamento
Unnamed: 0,1.000000,0.007333,-0.012281,0.014472,0.010891,0.011288,-0.035585,-0.103082
airbnb_listing_id,0.007333,1.000000,-0.050457,-0.038661,-0.012835,0.012875,-0.270643,0.082133
number_of_bedrooms,-0.012281,-0.050457,1.000000,0.758107,0.005798,-0.108500,-0.125340,0.131234
number_of_bathrooms,0.014472,-0.038661,0.758107,1.000000,0.020738,-0.103450,-0.103849,0.155047
star_rating,0.010891,-0.012835,0.005798,0.020738,1.000000,0.196364,0.121407,0.053912
is_superhost,0.011288,0.012875,-0.108500,-0.103450,0.196364,1.000000,0.287413,0.062671
number_of_reviews,-0.035585,-0.270643,-0.125340,-0.103849,0.121407,0.287413,1.000000,0.039613
faturamento,-0.103082,0.082133,0.131234,0.155047,0.053912,0.062671,0.039613,1.000000
